In [1]:
# LIMPIEZA DE DATOS CON PYSPARK

In [1]:
#importo la libreria
from pyspark.sql import SparkSession

In [2]:
#abro una sesion con un nombre
spark=SparkSession.builder.appName('pysarkpruebaslimpieza').getOrCreate()

In [3]:
# Creamos un DF con datos 
df = spark.createDataFrame([
        (1, 154.5, 45.9, 33, 'M'),
        (2, 167.2, 83.4, 45, 'M'),
        (3, 164.1, 61.2, 23, 'F'),
        (4, 162.5, 56.9, 23, 'M'),
        (5, 170.0, 65.7, 54, 'F'),
        (3, 164.1, 61.2, 23, 'F'),
        (5, 159.2, 59.3, 42, 'M'),
        (5, 159.2, 59.3, 42, 'F'),
        (6, 129.2, 35.3, None, 'M'),
        (7, 159.2, 59.3, 42, None),
        (8, None, None, 35, 'M'),
        (None, 159.2, 59.3, 42, 'M'),
        (5, 110.2, 23.4, None, 'F'),        
    ], ['id', 'altura', 'peso', 'edad', 'genero'])

In [4]:
df.show()

+----+------+----+----+------+
|  id|altura|peso|edad|genero|
+----+------+----+----+------+
|   1| 154.5|45.9|  33|     M|
|   2| 167.2|83.4|  45|     M|
|   3| 164.1|61.2|  23|     F|
|   4| 162.5|56.9|  23|     M|
|   5| 170.0|65.7|  54|     F|
|   3| 164.1|61.2|  23|     F|
|   5| 159.2|59.3|  42|     M|
|   5| 159.2|59.3|  42|     F|
|   6| 129.2|35.3|null|     M|
|   7| 159.2|59.3|  42|  null|
|   8|  null|null|  35|     M|
|null| 159.2|59.3|  42|     M|
|   5| 110.2|23.4|null|     F|
+----+------+----+----+------+



In [5]:
#verifico duplicados, la fila con id=3 es la misma, las demas son diferentes
print('Cantidad de Filas: ', df.count())
print('Cantidad de filas diferentes: ', df.distinct().count())

Cantidad de Filas:  13
Cantidad de filas diferentes:  12


In [6]:
# elimno duplicados
df = df.dropDuplicates()
df.show()
df.count()

+----+------+----+----+------+
|  id|altura|peso|edad|genero|
+----+------+----+----+------+
|null| 159.2|59.3|  42|     M|
|   3| 164.1|61.2|  23|     F|
|   5| 159.2|59.3|  42|     M|
|   1| 154.5|45.9|  33|     M|
|   4| 162.5|56.9|  23|     M|
|   2| 167.2|83.4|  45|     M|
|   8|  null|null|  35|     M|
|   5| 110.2|23.4|null|     F|
|   7| 159.2|59.3|  42|  null|
|   5| 159.2|59.3|  42|     F|
|   5| 170.0|65.7|  54|     F|
|   6| 129.2|35.3|null|     M|
+----+------+----+----+------+



12

In [7]:
# contar ids diferentes, no tuplas..en este caso 5
import pyspark.sql.functions as fn

df.agg(
    fn.count('id').alias('count'),
    fn.countDistinct('id').alias('distinct')
).show()
#fijense que no cuenta el id null

+-----+--------+
|count|distinct|
+-----+--------+
|   11|       8|
+-----+--------+



In [ ]:
# darle a cada fila un id unico
df.withColumn('nuevo_id', fn.monotonically_increasing_id()).show()

In [10]:
#cuantos no nulas tengo en edad?
print("la cantidad de NO nulos es: ", df.filter(df.edad.isNotNull()).count())

la cantidad de NO nulos es:  10


In [11]:
#a la columna edad cuando es nula ponerle la edad de 0
#importat la libreria primero
from pyspark.sql.functions import when

dfnuevo = df.withColumn("edad",when((df['edad'].isNull()), 0).otherwise(df['edad']))

In [12]:
# a ver si funciono?
print("la cantidad de NO nulos ahora en dfnuevo es: ", dfnuevo.filter(df.edad.isNotNull()).count())
dfnuevo.show()

la cantidad de NO nulos ahora en dfnuevo es:  10
+----+------+----+----+------+
|  id|altura|peso|edad|genero|
+----+------+----+----+------+
|null| 159.2|59.3|  42|     M|
|   3| 164.1|61.2|  23|     F|
|   5| 159.2|59.3|  42|     M|
|   1| 154.5|45.9|  33|     M|
|   4| 162.5|56.9|  23|     M|
|   2| 167.2|83.4|  45|     M|
|   8|  null|null|  35|     M|
|   5| 110.2|23.4|   0|     F|
|   7| 159.2|59.3|  42|  null|
|   5| 159.2|59.3|  42|     F|
|   5| 170.0|65.7|  54|     F|
|   6| 129.2|35.3|   0|     M|
+----+------+----+----+------+



In [13]:
#si queremos borrar las filas que tienen valores nulos
dfnuevo=df.na.drop()
dfnuevo.show()

+---+------+----+----+------+
| id|altura|peso|edad|genero|
+---+------+----+----+------+
|  3| 164.1|61.2|  23|     F|
|  5| 159.2|59.3|  42|     M|
|  1| 154.5|45.9|  33|     M|
|  4| 162.5|56.9|  23|     M|
|  2| 167.2|83.4|  45|     M|
|  5| 159.2|59.3|  42|     F|
|  5| 170.0|65.7|  54|     F|
+---+------+----+----+------+



In [14]:
#reemplazar los valores nulos 
dfnuevo=df.na.fill(400)
dfnuevo.show()

+---+------+-----+----+------+
| id|altura| peso|edad|genero|
+---+------+-----+----+------+
|400| 159.2| 59.3|  42|     M|
|  3| 164.1| 61.2|  23|     F|
|  5| 159.2| 59.3|  42|     M|
|  1| 154.5| 45.9|  33|     M|
|  4| 162.5| 56.9|  23|     M|
|  2| 167.2| 83.4|  45|     M|
|  8| 400.0|400.0|  35|     M|
|  5| 110.2| 23.4| 400|     F|
|  7| 159.2| 59.3|  42|  null|
|  5| 159.2| 59.3|  42|     F|
|  5| 170.0| 65.7|  54|     F|
|  6| 129.2| 35.3| 400|     M|
+---+------+-----+----+------+

